In [1]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [23]:
#retrieve data from the Swiss TXT API
base_football_url = "https://test.sport.api.swisstxt.ch/v1/contests/football/"

#define league
liga = "premier-league/"
#define season
season = "2022"
#define language
lang = "?lang=DE"


In [24]:
#get season
season_info_raw = requests.get(base_football_url + liga+lang).json()
data_season = season_info_raw['phases']

In [25]:
# Normalize the JSON data
normalized_data = json_normalize(data_season)

# Convert the normalized data to a Pandas DataFrame
df_season_raw = pd.DataFrame(normalized_data)

# View the DataFrame
df_season_raw.head()

/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_4296/331063180.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(data_season)


,id,key,displayName,state,isActual,showActuals,type,cesimReference.contestId,cesimReference.masterEventId,cesimReference.eventId,sport.id,sport.key,sport.name
0,4698-86-160,4698-86-160,1. Runde,Finished,False,False,Round,4698,86,160,7,football,Fussball
1,4698-86-161,4698-86-161,2. Runde,Finished,False,False,Round,4698,86,161,7,football,Fussball
2,4698-86-162,4698-86-162,3. Runde,Finished,False,False,Round,4698,86,162,7,football,Fussball
3,4698-86-163,4698-86-163,4. Runde,Finished,False,False,Round,4698,86,163,7,football,Fussball
4,4698-86-164,4698-86-164,5. Runde,Finished,False,False,Round,4698,86,164,7,football,Fussball


In [26]:
#get the season id (for now singular)
df_season_raw['season_id'] = df_season_raw['cesimReference.contestId'].astype(str) + "-" + df_season_raw['cesimReference.masterEventId'].astype(str)
season_ids = list(set(df_season_raw['season_id']))
season_id = season_ids[0]

In [27]:
#get all the games for this season
base_url_eventitems = "https://test.sport.api.swisstxt.ch/v1/eventItems?phaseIds="
season_id = season_ids[0]
start_date = "2016-01-01" #we don't care as long as it includes our chosen season above
end_date = "2030-01-01"
games_info_raw = requests.get(base_url_eventitems + season_id +"&fromDate="+start_date +"&toDate="+end_date+lang).json()
finished_rounds = list(df_season_raw[df_season_raw['state']=="Finished"]['displayName'])

In [42]:
games_info_raw[i]['contestInfo']['phases'][0]

{'id': '4698-86-160',
 'key': '4698-86-160',
 'name': {'de': '1. Runde', 'fr': '1ère journée', 'it': '1a giornata'},
 'categoryKey': 86,
 'type': 'Round'}

In [45]:
#extract the nested info about the phase, as the normalization below doesn't get this one
container = []

for i in range(0,len(games_info_raw)):
    try:
        game = {'id': games_info_raw[i]['id'],
                'round_id': games_info_raw[i]['contestInfo']['phases'][1]['id']}
        container.append(game)
    except:
        game = {'id': games_info_raw[i]['id'],
                'round_id': games_info_raw[i]['contestInfo']['phases'][0]['id']}
        container.append(game)

phase_info = pd.DataFrame(container)
    

In [46]:
# Normalize the JSON data
normalized_data = json_normalize(games_info_raw)

# Convert the normalized data to a Pandas DataFrame
df_games_raw = pd.DataFrame(normalized_data)


/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_4296/3859984260.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(games_info_raw)


In [47]:
#drop all the columns in Italian and French
cols_to_drop = [col for col in df_games_raw.columns if col.endswith('.it') or col.endswith('.fr')]
df_games_raw.drop(cols_to_drop, axis=1, inplace=True)
pd.set_option('display.max_columns', None)

df_games_raw = pd.merge(df_games_raw, phase_info, on='id')

df_games_raw.head()

,id,type,state,eventItemOrder,pushLiveData,dateTimeInfo.fullDateTime,dateTimeInfo.date,dateTimeInfo.time,displayTitle.de,sport.id,sport.key,sport.name.de,competitor1.id,competitor1.infostradaId,competitor1.name.de,competitor1.shortName.de,competitor1.country,competitor1.isNationalTeam,competitor1.results.main,competitor1.results.firstHalf,competitor1.results.secondHalf,competitor2.id,competitor2.infostradaId,competitor2.name.de,competitor2.shortName.de,competitor2.country,competitor2.isNationalTeam,competitor2.results.main,competitor2.results.firstHalf,competitor2.results.secondHalf,contestInfo.contestSeason.id,contestInfo.contestSeason.key,contestInfo.contestSeason.name.de,contestInfo.contestSeason.season,contestInfo.phases,scores.firstHalf.competitor1,scores.firstHalf.competitor2,scores.firstHalf.formatted.de,scores.secondHalf.competitor1,scores.secondHalf.competitor2,scores.secondHalf.formatted.de,scores.main.competitor1,scores.main.competitor2,scores.main.formatted.de,city.de,stadium.de,spectators.de,referees.de,refereeCountry.de,round_id
0,1750685,Game,Finished,0,True,2022-08-05T21:00:00,Fr 05.08.,21:00,Crystal Palace - Arsenal,7,football,Fussball,13541,100094,Crystal Palace,Crystal P.,ENG,False,0.0,0.0,0.0,5457,4007,Arsenal,Arsenal,ENG,False,2.0,1.0,1.0,4698-86,premier-league,Premier League,2023,"[{'id': '4698-86-160', 'key': '4698-86-160', '...",0.0,1.0,0 : 1,0.0,1.0,0 : 1,0.0,2.0,0 : 2,London,Selhurst Park,25286,Anthony Taylor,ENG,4698-86-160
1,1750686,Game,Finished,0,True,2022-08-06T13:30:00,Sa 06.08.,13:30,Fulham - Liverpool,7,football,Fussball,5465,102785,Fulham,Fulham,ENG,False,2.0,1.0,1.0,5454,4075,Liverpool,Liverpool,ENG,False,2.0,0.0,2.0,4698-86,premier-league,Premier League,2023,"[{'id': '4698-86-160', 'key': '4698-86-160', '...",1.0,0.0,1 : 0,1.0,2.0,1 : 2,2.0,2.0,2 : 2,London,Craven Cottage,22207,Andy Madley,ENG,4698-86-160
2,1750687,Game,Finished,0,True,2022-08-06T16:00:00,Sa 06.08.,16:00,Tottenham Hotspur - Southampton,7,football,Fussball,5460,4129,Tottenham Hotspur,Tottenham,ENG,False,4.0,2.0,2.0,5451,4874,Southampton,Southampton,ENG,False,1.0,1.0,0.0,4698-86,premier-league,Premier League,2023,"[{'id': '4698-86-160', 'key': '4698-86-160', '...",2.0,1.0,2 : 1,2.0,0.0,2 : 0,4.0,1.0,4 : 1,London,Tottenham Hotspur Stadium,61732,André Marriner,ENG,4698-86-160
3,1750688,Game,Finished,0,True,2022-08-06T16:00:00,Sa 06.08.,16:00,Newcastle - Nottingham Forest,7,football,Fussball,5463,4087,Newcastle,Newcastle,ENG,False,2.0,0.0,2.0,17751,4088,Nottingham Forest,Nottingham,ENG,False,0.0,0.0,0.0,4698-86,premier-league,Premier League,2023,"[{'id': '4698-86-160', 'key': '4698-86-160', '...",0.0,0.0,0 : 0,2.0,0.0,2 : 0,2.0,0.0,2 : 0,Newcastle,St James' Park,52245,Simon Hooper,ENG,4698-86-160
4,1750690,Game,Finished,0,True,2022-08-06T16:00:00,Sa 06.08.,16:00,Leeds United - Wolverhampton,7,football,Fussball,5450,4073,Leeds United,Leeds Utd,ENG,False,2.0,1.0,1.0,8995,4211,Wolverhampton,Wolverham.,ENG,False,1.0,1.0,0.0,4698-86,premier-league,Premier League,2023,"[{'id': '4698-86-160', 'key': '4698-86-160', '...",1.0,1.0,1 : 1,1.0,0.0,1 : 0,2.0,1.0,2 : 1,Leeds,Elland Road,36347,Robert Jones,ENG,4698-86-160


In [48]:
#get a list of all the teams of this season
all_teams = list(set(df_games_raw["competitor1.name.de"]))

In [66]:
# Create a list of dictionaries for each finished game with the round ID, competitor names, and scores
games = []
for i in range(len(df_games_raw)):
    if df_games_raw.loc[i, 'state'] == 'Finished':
        game = {}
        game['round_id'] = df_games_raw.loc[i, 'round_id']
        game['team1'] = df_games_raw.loc[i, 'competitor1.name.de']
        game['team2'] = df_games_raw.loc[i, 'competitor2.name.de']
        game['team1_score'] = df_games_raw.loc[i, 'competitor1.results.main']
        game['team2_score'] = df_games_raw.loc[i, 'competitor2.results.main']
        games.append(game)

# Create a list of dictionaries for each team with the team name, round ID, points, goals scored, and goals scored against
teams = []
for team in all_teams:
    for i, round_id in enumerate(df_games_raw['round_id'].unique(), start=1):
        team_data = {'team': team, 
                     'round_id': round_id, 
                     'points': 0, 'goals_scored': 0, 
                     'goals_conceded': 0, 
                     'round_num': i}
        teams.append(team_data)

# Calculate the points and other stats for each team in each round based on their scores in the games
for game in games:
    team1_goals = game['team1_score']
    team2_goals = game['team2_score']
    team1_points = 1
    team2_points = 1
    if team1_goals > team2_goals:
        team1_points = 3
        team2_points = 0
    elif team1_goals < team2_goals:
        team1_points = 0
        team2_points = 3
    for team in teams:
        if team['team'] == game['team1'] and team['round_id'] == game['round_id']:
            team['points'] += team1_points
            team['goals_scored'] += team1_goals
            team['goals_conceded'] += team2_goals
        elif team['team'] == game['team2'] and team['round_id'] == game['round_id']:
            team['points'] += team2_points
            team['goals_scored'] += team2_goals
            team['goals_conceded'] += team1_goals

# Create a new DataFrame from the list of team dictionaries
df_stats = pd.DataFrame(teams)
df_stats

,team,round_id,points,goals_scored,goals_conceded,round_num
0,Manchester City,4698-86-160,3,2.0,0.0,1
1,Manchester City,4698-86-161,3,4.0,0.0,2
2,Manchester City,4698-86-162,1,3.0,3.0,3
3,Manchester City,4698-86-163,3,4.0,2.0,4
4,Manchester City,4698-86-164,3,6.0,0.0,5
...,...,...,...,...,...,...
755,Liverpool,4698-86-193,0,0.0,0.0,34
756,Liverpool,4698-86-194,0,0.0,0.0,35
757,Liverpool,4698-86-195,0,0.0,0.0,36
758,Liverpool,4698-86-291,0,0.0,0.0,37


In [50]:
# Add up points for each team one round after another
df_stats.sort_values(['team', 'round_num'], inplace=True)
df_stats['total_points'] = df_stats.groupby('team')['points'].cumsum()

# Calculate average points scored for last 5 rounds
df_stats['avg_points_last_5'] = df_stats.groupby('team')['points'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average points scored for last 10 rounds
df_stats['avg_points_last_10'] = df_stats.groupby('team')['points'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 5 rounds
df_stats['avg_goals_scored_last_5'] = df_stats.groupby('team')['goals_scored'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 10 rounds
df_stats['avg_goals_scored_last_10'] = df_stats.groupby('team')['goals_scored'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 5 rounds
df_stats['avg_goals_conceded_last_5'] = df_stats.groupby('team')['goals_conceded'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 10 rounds
df_stats['avg_goals_conceded_last_10'] = df_stats.groupby('team')['goals_conceded'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Sort by round_num
df_stats.sort_values(['team','round_num'], inplace=True)
df_stats = df_stats.reset_index()


In [51]:
df_season_small = df_season_raw[["id", "displayName"]]
df_season_small.columns = ["round_id", "displayName"]
df_stats = pd.merge(df_stats, df_season_small)


In [52]:
# Number of rounds left
df_stats['num_rounds_left'] = max(df_stats['round_num'])-df_stats['round_num']
df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left
0,456,Arsenal,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37
1,38,Aston Villa,4698-86-160,0,0.0,2.0,1,0,0.0,0.0,0.0,0.0,2.0,2.0,1. Runde,37
2,380,Bournemouth,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37
3,76,Brentford,4698-86-160,1,2.0,2.0,1,1,1.0,1.0,2.0,2.0,2.0,2.0,1. Runde,37
4,342,Brighton,4698-86-160,3,2.0,1.0,1,3,3.0,3.0,2.0,2.0,1.0,1.0,1. Runde,37


In [53]:
#get the projected averages
proj_points_avg_last_5 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_5'])
df_stats['proj_points_avg_last_5'] = proj_points_avg_last_5
proj_points_avg_last_10 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_10'])
df_stats['proj_points_avg_last_10'] = proj_points_avg_last_10
df_stats.head()


,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10
0,456,Arsenal,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0
1,38,Aston Villa,4698-86-160,0,0.0,2.0,1,0,0.0,0.0,0.0,0.0,2.0,2.0,1. Runde,37,0.0,0.0
2,380,Bournemouth,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0
3,76,Brentford,4698-86-160,1,2.0,2.0,1,1,1.0,1.0,2.0,2.0,2.0,2.0,1. Runde,37,38.0,38.0
4,342,Brighton,4698-86-160,3,2.0,1.0,1,3,3.0,3.0,2.0,2.0,1.0,1.0,1. Runde,37,114.0,114.0


In [54]:
# Calculate goal difference for each game
df_stats['goal_diff_game'] = df_stats['goals_scored'] - df_stats['goals_conceded']

# Group by round and sum goal differences
df_stats['goal_diff'] = df_stats.groupby('team')['goal_diff_game'].cumsum()

#Rank now for each round
df_stats['rank'] = df_stats.sort_values(by=['total_points', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats['rank_proj5'] = df_stats.sort_values(by=['proj_points_avg_last_5', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats['rank_proj10'] = df_stats.sort_values(by=['proj_points_avg_last_10', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1

df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff,rank,rank_proj5,rank_proj10
0,456,Arsenal,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0,2.0,2.0,2,2,2
1,38,Aston Villa,4698-86-160,0,0.0,2.0,1,0,0.0,0.0,0.0,0.0,2.0,2.0,1. Runde,37,0.0,0.0,-2.0,-2.0,16,16,16
2,380,Bournemouth,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0,2.0,2.0,3,3,3
3,76,Brentford,4698-86-160,1,2.0,2.0,1,1,1.0,1.0,2.0,2.0,2.0,2.0,1. Runde,37,38.0,38.0,0.0,0.0,9,9,9
4,342,Brighton,4698-86-160,3,2.0,1.0,1,3,3.0,3.0,2.0,2.0,1.0,1.0,1. Runde,37,114.0,114.0,1.0,1.0,6,6,6


In [55]:
df_stats[['rank_proj10','team']][df_stats['displayName']=="29. Runde"].sort_values('rank_proj10')

,rank_proj10,team
560,1,Arsenal
572,2,Manchester City
573,3,Manchester United
574,4,Newcastle
577,5,Tottenham Hotspur
561,6,Aston Villa
564,7,Brighton
563,8,Brentford
571,9,Liverpool
565,10,Chelsea


In [18]:
df_stats[df_stats['displayName'].isin(finished_rounds)].to_csv('bundesliga.csv', index=False)

In [56]:
df_stats

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff,rank,rank_proj5,rank_proj10
0,456,Arsenal,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0,2.0,2.0,2,2,2
1,38,Aston Villa,4698-86-160,0,0.0,2.0,1,0,0.0,0.0,0.0,0.0,2.0,2.0,1. Runde,37,0.0,0.0,-2.0,-2.0,16,16,16
2,380,Bournemouth,4698-86-160,3,2.0,0.0,1,3,3.0,3.0,2.0,2.0,0.0,0.0,1. Runde,37,114.0,114.0,2.0,2.0,3,3,3
3,76,Brentford,4698-86-160,1,2.0,2.0,1,1,1.0,1.0,2.0,2.0,2.0,2.0,1. Runde,37,38.0,38.0,0.0,0.0,9,9,9
4,342,Brighton,4698-86-160,3,2.0,1.0,1,3,3.0,3.0,2.0,2.0,1.0,1.0,1. Runde,37,114.0,114.0,1.0,1.0,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,531,Nottingham Forest,4698-86-292,0,0.0,0.0,38,27,0.0,0.1,0.0,0.3,0.0,0.8,38. Runde,0,27.0,27.0,0.0,-33.0,19,19,19
756,303,Southampton,4698-86-292,0,0.0,0.0,38,24,0.0,0.1,0.0,0.4,0.0,1.0,38. Runde,0,24.0,24.0,0.0,-29.0,20,20,20
757,455,Tottenham Hotspur,4698-86-292,0,0.0,0.0,38,53,0.0,0.4,0.0,0.6,0.0,1.1,38. Runde,0,53.0,53.0,0.0,7.0,6,6,6
758,189,West Ham United,4698-86-292,0,0.0,0.0,38,34,0.0,1.0,0.0,0.8,0.0,0.2,38. Runde,0,34.0,34.0,0.0,-8.0,14,14,14


In [57]:
df_games_raw['state'] == 'Finished'

0       True
1       True
2       True
3       True
4       True
       ...  
375    False
376    False
377    False
378    False
379    False
Name: state, Length: 380, dtype: bool

In [59]:
game['game_status']


'Finished'